In [17]:
import pandas as pd
from sqlalchemy import create_engine
import os

# os.chdir('/Users/macbook/Desktop/Data-Program-Files/Group-Projects/Project-2/Sport-Salaries/')


In [18]:
### Extract CSVs into DataFrames
NFL_file = "nfl_salary_2010_2020.csv"
NFL_df = pd.read_csv(NFL_file)
NFL_df.head()


,PLAYER,POS,TEAM,AGE,AVERAGE SALARY,TEAM AAV %,CASH EARNINGS,% OF TEAM PAYROLL,GTD EARNINGS,TEAM GTD %,CAP FIGURE,% OF TEAM CAP,Year
0,Tom Brady,QB,NE,33,18000000,0.17,16500000,0.2,-,-,20320000,-,2010
1,DeAngelo Hall,CB,WAS,26,9166667,0.06,18500000,0.14,-,-,19700000,-,2010
2,Peyton Manning,QB,IND,34,14171429,0.12,15800000,0.16,-,-,19266666,-,2010
3,Champ Bailey,CB,DEN,32,9000000,0.08,16500000,0.19,-,-,18895000,-,2010
4,Donovan McNabb,QB,WAS,33,14850000,0.1,18575000,0.14,-,-,18575000,-,2010


In [24]:
# Create a filtered dataframe from specific columns
nfl_cols = ["PLAYER", "POS", "TEAM", "AGE", "AVERAGE SALARY", "Year"]
nfl_transformed= NFL_df[nfl_cols].copy()

# Rename the column headers
nfl_transformed = nfl_transformed.rename(columns={"PLAYER": "player_name",
                                                          "POS": "position",
                                                          "TEAM": "team",
                                                          "AGE": "age",
                                                          "AVERAGE SALARY": "avg_salary",
                                                          "Year": "year"})

# Clean the data by dropping NA's, clean avg_salary, and setting the index
nfl_transformed = nfl_transformed.dropna(axis = 0, how ='any') 
nfl_transformed = nfl_transformed.sort_values('team', ascending=False).drop_duplicates(subset=['player_name', 'position', 'year'])

nfl_transformed_cond = nfl_transformed['avg_salary'] != "-"
nfl_transformed = nfl_transformed[nfl_transformed_cond]

nfl_transformed.set_index("year", inplace=True)
nfl_transformed.sort_values(by=['year'])
nfl_transformed.head(30)


,player_name,position,team,age,avg_salary
year,,,,,
2016,Colt McCoy,QB,WAS,29,3000000
2012,Domonique Johnson,CB,WAS,26,665000
2010,Rex Grossman,QB,WAS,30,755000
2020,Nick Sundberg,LS,WAS,33,1100000
2018,Adonis Alexander,CB,WAS,21,654048
2015,Kedric Golston,DT,WAS,32,1100000
2016,Preston Smith,OLB,WAS,23,1444839
2019,Hale Hentges,TE,WAS,23,590000
2018,Shaun-Dion Hamilton,ILB,WAS,22,653253


In [20]:
#connect to postgres
connection_string = "postgres:postgres@localhost:5432/nfl_db"
engine = create_engine(f'postgresql://{connection_string}')


In [15]:
# #create NFL player table. Says there is an error, but table has been created.
# pd.read_sql_query('CREATE TABLE nfl_player_salary(year INT, player_name VARCHAR(500), position VARCHAR(100), team VARCHAR(100), age INT, avg_salary INT, PRIMARY KEY (year, player_name, position))', con=engine).head()



In [21]:
# Confirm table
engine.table_names()


['nfl_player_salary']

In [22]:
#Load into NFL player Salary
nfl_transformed.to_sql(name='nfl_player_salary', con=engine, if_exists='append', index=True)


In [23]:
pd.read_sql_query('select * from nfl_player_salary', con=engine).head(30)


,year,player_name,position,team,age,avg_salary
0,2016,Colt McCoy,QB,WAS,29,3000000
1,2012,Domonique Johnson,CB,WAS,26,665000
2,2010,Rex Grossman,QB,WAS,30,755000
3,2020,Nick Sundberg,LS,WAS,33,1100000
4,2018,Adonis Alexander,CB,WAS,21,654048
5,2015,Kedric Golston,DT,WAS,32,1100000
6,2016,Preston Smith,OLB,WAS,23,1444839
7,2019,Hale Hentges,TE,WAS,23,590000
8,2018,Shaun-Dion Hamilton,ILB,WAS,22,653253
9,2012,Keenan Robinson,OLB,WAS,23,630027
